In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
import requests
import time
from time import sleep
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd
import os
from datetime import datetime

options = Options()
options.add_experimental_option("detach", True)
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
current_time = datetime.now().strftime('%y-%m-%d')

#네이버 지도 화면의 URL
url = "https://map.naver.com/v5/search"

driver.get(url)

time.sleep(3)
key_word = '덕수궁 음식점'


# 해당 요소 찾을때 까지 10초대기
def time_wait(num, code):
    try:
        wait = WebDriverWait(driver, num).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, code)))
    except:
        print(code, '태그를 찾지 못하였습니다.')
        driver.quit()
    return wait
    
# frame 변경 메소드
def switch_frame(frame):
    driver.switch_to.default_content()  # frame 초기화
    driver.switch_to.frame(frame)  # frame 변경

# 현재 스크래핑된 HTML 요소 스냅샷 만드는 함수
def save_html_snapshot(page_str, file_prefix):
    current_date = datetime.now().strftime('%y%m%d_%H%M%S')
    
    # 파일 경로 생성
    file_path = f'{file_prefix}_{current_date}.html'
    
    if not os.path.exists(file_path):
        soup = BeautifulSoup(page_str, 'html.parser')
        # 파일에 저장
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(soup.prettify())
        
        print("HTML 파일이 저장되었습니다.")

# 해당 요소를 찾을때 까지 10초 대기
time_wait(10, 'div.input_box > input.input_search')
# (1) 검색창 찾기
search = driver.find_element(By.CSS_SELECTOR, 'div.input_box > input.input_search')
search.send_keys(key_word)  # 검색어 입력
search.send_keys(Keys.ENTER)  # 엔터버튼 누르기

sleep(1)

# (2) frame 변경
switch_frame('searchIframe')
sleep(2)

# 이후 요소 찾기
element = driver.find_elements(By.CSS_SELECTOR, ".place_bluelink.TYaxT")
element[7].click()
sleep(2)

switch_frame('entryIframe')
sleep(2)

driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[4]/div/div/div/div/a[4]/span').click()
sleep(3)

index = 1
LIMIT_COUNT = 20
# LIMIT_COUNT = 30
# LIMIT_COUNT = 100  
# 메모리 상황에 따라 특정 LIMIT 수가 넘어 가면서 시스템이 다운될 수 있다.
sleep(1)
while True:
    try:
        if index == LIMIT_COUNT: # 더보기 9번을 수행하게 됨, 그럼 기존 10개에서 최대 100개의 리뷰를 얻을 수 있음
            break
        # driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a/span').click()
                                                   
        element = driver.find_element(By.CSS_SELECTOR, 'span.TeItc')
        # element.click() 이렇게 처리시 화면을 벗어난 경우 동작하지 않을 수 있다
        driver.execute_script("arguments[0].click();", element)
      
        print(f'방문자리뷰 페이지 더보기 클릭 {index}회 진행')
        index+=1
        # sleep(2)
        driver.implicitly_wait(10) # HTML 요소가 모두 로드되면 즉각 다음행으로 이동, 못찾으면 최대 10초까지 대기
        
    except:
        print('더이상 더보기가 없습니다.')
        break
print('방문자리뷰 페이지 더보기 마지막까지 검색 완료')  

# 모든 더보기를 찾습니다.
print('더보기 찾기')
# more_reviews = soup.find_all("span", class_="rvCSr")
# more_reviews = driver.find_elements(By.CSS_SELECTOR, "span.rvCSr")
# more_reviews = driver.find_elements(By.CSS_SELECTOR, "a.pui__xtsQN-")
more_reviews = driver.find_elements(By.CSS_SELECTOR, "a.pui__wFzIYl")
# pui__wFzIYl
index = 1
for long_review in more_reviews:
    print(f'{index} long_review: {long_review}, type: {type(long_review)}')
    # if long_review is not None:
    # long_review.click() # 화면에 벗어나 클릭 할 수 없다면 ElementClickInterceptedException 발생
    # long_review.click()
    try:
        # 화면에서 벗어난 더보기가 있는 리뷰에 대해서 자바스크립트를 통해 강제로 클릭
        driver.execute_script("arguments[0].click();", long_review) 
        sleep(2)
        driver.implicitly_wait(10)
        index = index + 1
        print(f'방문자리뷰 개별 리뷰 더보기 클릭 {index}회 진행')
    except ElementClickInterceptedException as e:
        print('요소를 클릭할 수 없습니다')
sleep(3)

reviews = []

# 다시 페이지 로드
soup=BeautifulSoup(driver.page_source,'html.parser')

# 모든 댓글 요소를 찾습니다.
# spans = soup.find_all("span", class_="zPfVt")
spans = soup.find_all("a", class_="pui__xtsQN-")

for span in spans:
    reviews.append(span.text)
    
df = pd.DataFrame(reviews, columns=["Review"])
df

file_name = f'네이버_리뷰_{current_time}.xlsx'
df.to_excel(file_name)

# driver.close()

방문자리뷰 페이지 더보기 클릭 1회 진행
방문자리뷰 페이지 더보기 클릭 2회 진행
방문자리뷰 페이지 더보기 클릭 3회 진행
방문자리뷰 페이지 더보기 클릭 4회 진행
방문자리뷰 페이지 더보기 클릭 5회 진행
방문자리뷰 페이지 더보기 클릭 6회 진행
방문자리뷰 페이지 더보기 클릭 7회 진행
방문자리뷰 페이지 더보기 클릭 8회 진행
방문자리뷰 페이지 더보기 클릭 9회 진행
방문자리뷰 페이지 더보기 클릭 10회 진행
방문자리뷰 페이지 더보기 클릭 11회 진행
방문자리뷰 페이지 더보기 클릭 12회 진행
방문자리뷰 페이지 더보기 클릭 13회 진행
방문자리뷰 페이지 더보기 클릭 14회 진행
방문자리뷰 페이지 더보기 클릭 15회 진행
방문자리뷰 페이지 더보기 클릭 16회 진행
방문자리뷰 페이지 더보기 클릭 17회 진행
방문자리뷰 페이지 더보기 클릭 18회 진행
방문자리뷰 페이지 더보기 클릭 19회 진행
방문자리뷰 페이지 더보기 마지막까지 검색 완료


더보기 찾기
1 long_review: <selenium.webdriver.remote.webelement.WebElement (session="7685bc7f8a8adb3b0fb627f93bb6da36", element="f.1704D25678EC26683109D22CDE6462C0.d.8F721E4FEFFFEA32D108180D21D17112.e.458")>, type: <class 'selenium.webdriver.remote.webelement.WebElement'>
방문자리뷰 개별 리뷰 더보기 클릭 2회 진행
2 long_review: <selenium.webdriver.remote.webelement.WebElement (session="7685bc7f8a8adb3b0fb627f93bb6da36", element="f.1704D25678EC26683109D22CDE6462C0.d.8F721E4FEFFFEA32D108180D21D17112.e.459")>, type: <class 'selenium.webdriver.remote.webelement.WebElement'>
방문자리뷰 개별 리뷰 더보기 클릭 3회 진행
3 long_review: <selenium.webdriver.remote.webelement.WebElement (session="7685bc7f8a8adb3b0fb627f93bb6da36", element="f.1704D25678EC26683109D22CDE6462C0.d.8F721E4FEFFFEA32D108180D21D17112.e.460")>, type: <class 'selenium.webdriver.remote.webelement.WebElement'>
방문자리뷰 개별 리뷰 더보기 클릭 4회 진행
4 long_review: <selenium.webdriver.remote.webelement.WebElement (session="7685bc7f8a8adb3b0fb627f93bb6da36", element="f.1704D25678EC

,Review
0,"광화문역 에서가까운 내수동 커리맛집, 스페셜한 카레가먹고 싶어온 고가빈커리하우스...."
1,"🏅🏅🏅♡쉬림프시금치커리 (15,000)♡플레인로띠 (3,000)♥맛-쉬림프시금치커리..."
2,오믈렛커리(버터).(시금치). 버터치킨커리. 쉬림프시금치커리. 에크카츠카레그리거 코...
3,세번째 방문이네용 가끔씩 생각나는 맛이에요쉬림프는 처음 먹었는데 역시 맛있구요 사이...
4,커리 먹고싶을 때마다 오는 곳이에요! 멀어서 자주는못와도 연에 두번씩은 와요🤣 개인...
5,여기는 진짜 맛있어요! 그리고 메인메뉴주문시 밥과 커리소스가 리필이 1회 되더라구요...
6,카레 너무 맛있어요😍개인적으로 시금치카레보다는 버터카레가 더 진하고 맛있었어요신랑은...
7,오픈 시간 전에 가서 기다리다가1번으로 들어갔어요~저는 특히 시금치 커리랑 플레인 ...
8,양이 많아요. 소스랑 밥도 리필되는데 배불러서 소스만 한번 더 리필했어요.. 오믈렛...
9,"🍽️ 버터치킨커리, 오믈렛커리(시금치), 코코넛로띠✔️ 커리와 밥은 각각 리필이 가..."


In [708]:
df.iloc[0]['Review']

'광화문역 에서가까운 내수동 커리맛집, 스페셜한 카레가먹고 싶어온  고가빈커리하우스.  커리맛이진하고 독특해서 한번씩  여행가고싶을때 맛보러오곤해요, 일본식 오믈렛커리는 반숙되어진 촉촉한 계란오므렛을 진한카레맛과함께 조금씩떠서 먹으면 정말 일본여행가서 먹던 맛이예요, 쉬림프 시금치커리는 매콤한맛과 진한 카레와 잘어우러져 인도커리맛을 느끼게해요, 치킨가라아케를 추가로시켜서 카레와함께 먹으면 더 다양한맛이나죠, 코코넛로띠는 달콤하고 부드러운 찰진맛이 그냥 먹어도 맛있고 카레에 살짝찍어먹어도 맛있어요, 외국인들에게 인기가많아서 외국인 가족분들이 와서 식사를 하고 있네요, 글로벌한 카레맛집이네요'